In [ ]:
! pip install -U -q "google-genai>=0.7.2"

In [ ]:

import os
from dotenv import load_dotenv  # helps to load env files


load_dotenv()

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
print("GOOGLE_API_KEY : : : ", GOOGLE_API_KEY)

In [ ]:
import google.generativeai as genai

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
def enable_light():
    """Turn on the lighting system"""
    print("LIGHTBOT : Lights Enabled")

def set_light_color(rgb_hex:str):
    """Set the light color . List must be enabled for this tow work"""
    print(f"LIGHTBOT : Lights set to {rgb_hex}.")


def stop_light():
    """stop flashing light"""
    print(f"LIGHTBOT : Lights turned off .")


In [ ]:

light_controls = [enable_light , set_light_color , stop_light]

instruction = """You are a helpful lighting system bot. You can trun lights on and off ,
                 and you can set the light color . Do not perform any other task"""

model = genai.GenerativeModel(
    "gemini-2.0-flash", 
    tools=light_controls , 
    system_instruction=instruction
)

chat = model.start_chat()

# This tells us no of tools assigned to particular model
light_controls , model , chat , model._tools.to_proto()

### Create helper function for setting function_calling_config on tool_config

In [ ]:
from google.generativeai.types import content_types
from collections.abc import Iterable

# mode can be of 3 types 
# none -> 
# auto -> 
# any -> 

def tool_config_from_mode(mode:str , fns:Iterable[str] = ()):
    """"Create a tool config with the specified function calling mode """,
    return content_types.to_tool_config(
        {"function_calling_config":{"mode":mode , "allowed_function_names":fns}}
    )

In [ ]:
tool_config = tool_config_from_mode("none")

In [ ]:
response = chat.send_message("Hello light bot , what can you do", tool_config=tool_config)

response

In [ ]:
tool_config = tool_config_from_mode("auto")

tool_config

response = chat.send_message("light this place up ", tool_config=tool_config)

response.parts[0]

# you are not actually calling the function , so remove this from the history .
# chat.rewind()

In [ ]:
available_fns = ['set_light_color', 'enable_light']

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("make this place purple and light this place up as well ", tool_config=tool_config)

response.parts


### Automatic Function Calling

##### Our Agentic Ai will decide own itself what funct need to be execute based on given prompt. 

In [ ]:
available_fns = ['set_light_color', 'enable_light']
tool_config = tool_config_from_mode("any", available_fns)


auto_chat = model.start_chat(enable_automatic_function_calling=True)
response = auto_chat.send_message("make this place purple and light this place up as well", tool_config=tool_config)

In [ ]:
response.parts[0].text

In [ ]:
available_fns = ['set_light_color', 'enable_light' , 'stop_light']
tool_config = tool_config_from_mode("any", available_fns)


auto_chat = model.start_chat(enable_automatic_function_calling=True)
response = auto_chat.send_message("It's awful dark in here - make it orange color", tool_config=tool_config)

In [ ]:
response.parts[0].text

### Parallel function calling
##### In addition to basic function calling described above, you can also call multiple functions in a single turn. 

In [ ]:
available_fns = ['set_light_color', 'enable_light' , 'stop_light']
tool_config = tool_config_from_mode("any", available_fns)


auto_chat = model.start_chat(enable_automatic_function_calling=True)
response = auto_chat.send_message("It's awful dark in here - make it blue color and then switch off the light", tool_config=tool_config)

In [ ]:
response.parts[0].text

In [ ]:
available_fns = ['set_light_color', 'enable_light' , 'stop_light']
tool_config = tool_config_from_mode("any", available_fns)


auto_chat = model.start_chat(enable_automatic_function_calling=True)
response = auto_chat.send_message("switch off the light", tool_config=tool_config)

### Performing Automatic tool calling on another example 

In [ ]:
def set_light_values(brightness: int, color_temp: str) -> dict[str, int | str]:
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {
        "brightness": brightness,
        "colorTemperature": color_temp
    }

In [ ]:
model = genai.GenerativeModel(model_name="gemini-2.0-flash", tools=[set_light_values])
model

In [ ]:
chat = model.start_chat(enable_automatic_function_calling=True)


response = chat.send_message('Dim the light so the room feels cozy and warm')
print(response.parts[0].text)

### Parallel function calling
##### In addition to basic function calling described above, you can also call multiple functions in a single turn. 

In [ ]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

In [ ]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]


model = genai.GenerativeModel(model_name="gemini-2.0-flash" , tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message('Turn this place into a party night!')

# Print out each of the function calls requested from this single call.
for part in response.parts:
  if fn := part.function_call:
    args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
    print(f"{fn.name}({args})")

In [ ]:
# Call the API.
chat = model.start_chat()
response = chat.send_message('Turn this place into sleepy night!')

# Print out each of the function calls requested from this single call.
for part in response.parts:
  if fn := part.function_call:
    args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
    print(f"{fn.name}({args})")

In [ ]:
# This tells us no of tools assigned to particular model
model._tools.to_proto()